In [1]:
from copy import deepcopy
from itertools import combinations
import pickle
import typing as tp
from zipfile import ZipFile

from lightfm import LightFM
from lightfm.data import Dataset as LFMDataset
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.preprocessing import normalize
from transliterate import translit

In [2]:
PATH_TO_DATA = "../data_kion.zip"
RANDOM_STATE = 42

## Загрузка данных

Для целей семинара используем анонимизированные данные по просмотрам в онлайн-кинотеатре Кион. Мы имеем данные по действиям примерно за полгода, а также признаки пользователей и контента.

In [3]:
with ZipFile(PATH_TO_DATA) as zf:
    with zf.open("interactions.csv") as f:
        interactions = pd.read_csv(f)
    
    with zf.open("items.csv") as f:
        items = pd.read_csv(f)
    

In [4]:
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [5]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,keywords,description,business_dt
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...","Поговори, ней, 2002, Испания, друзья, любовь, ...",Мелодрама легендарного Педро Альмодовара «Пого...,2021-09-10
1,8250,film,Объезд,Detour,2013.0,"зарубежные, триллеры",США,NaN,16.0,NaN,Уильям Дикерсон,"Бриа Грант, Джон Форест, Дэб Снайдер, Нил Хопк...","Объезд, 2013, США, выживание, диких, условиях,...","История среднестатистического американца, чью ...",2021-09-10
2,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...","Голые, перцы, 2014, США, друзья, свадьбы, прео...",Уморительная современная комедия на популярную...,2021-09-10
3,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...","Тактическая, сила, 2011, Канада, бандиты, ганг...",Профессиональный рестлер Стив Остин («Все или ...,2021-09-10
4,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","45, лет, 2015, Великобритания, брак, жизнь, лю...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...",2021-09-10


Определим последний доступный день в выборке как тестовый, а 3 месяца до него - обучающей выборкой.

Для простоты:
- удачным взаимодействием будем считать просмотр любой длительности и при обучении будем использовать только эту информацию, игнорируя данные о признаках пользователей и контента
- не будем обрабатывать "холодных" пользователей, просто удалим их из тестовой выборки

In [6]:
train = interactions.loc[
    (interactions["last_watch_dt"] >= "2021-05-21")
    & (interactions["last_watch_dt"] <= "2021-08-21"),
    ["user_id", "item_id"]
]

test = interactions.loc[interactions["last_watch_dt"] == "2021-08-22", ["user_id", "item_id"]]
test = test.loc[test["user_id"].isin(train["user_id"]) & test["item_id"].isin(train["item_id"])]

In [7]:
train = pd.merge(train, items[["item_id", "title", "genres"]], on="item_id")
test = pd.merge(test, items[["item_id", "title", "genres"]], on="item_id")

## Добавление аватаров в обучающую выборку

Далее для построения рекомендаций будем использовать алгоритм матричного разложения из библиотеки LightFM. Для формирования рекомендаций для аватаров необходимо, чтобы действия аватаров были в обучающей выборке.

Отметим, что добавляя действия аватаров в обучающую выборку, мы несколько изменяем ее распределение. Обычно этим можно пренебречь, т.к. объем обучающей выборки (в нашем случае 5 млн наблюдений) значительно больше объема действий аватаров (10 наблюдений).

In [8]:
titles = [
    "Джон Уик", 
    "Заложница", 
    "Перевозчик",
    "Форсаж: Хоббс и Шоу",
    "Терминатор 3: Восстание машин"
]
avatar_interactions_action = pd.DataFrame({"user_id": "avatar_action", "title": titles})
avatar_interactions_action = avatar_interactions_action.merge(items[["item_id", "title", "genres"]], on="title")
avatar_interactions_action

,user_id,title,item_id,genres
0,avatar_action,Джон Уик,7671,"боевики, триллеры"
1,avatar_action,Заложница,14362,"боевики, триллеры"
2,avatar_action,Перевозчик,8350,"боевики, триллеры, криминал"
3,avatar_action,Форсаж: Хоббс и Шоу,2323,"боевики, триллеры"
4,avatar_action,Терминатор 3: Восстание машин,2925,"боевики, фантастика, триллеры"


In [9]:
titles = [
    "Тупой и еще тупее 2",
    "Типа крутые легавые",
    "Голый пистолет",
    "Убойные каникулы",
    "Карты, деньги, два ствола"
]
avatar_interactions_comedy = pd.DataFrame({"user_id": "avatar_comedy", "title": titles})
avatar_interactions_comedy = avatar_interactions_comedy.merge(items[["item_id", "title", "genres"]], on="title")
avatar_interactions_comedy

,user_id,title,item_id,genres
0,avatar_comedy,Тупой и еще тупее 2,8391,комедии
1,avatar_comedy,Типа крутые легавые,2915,"боевики, триллеры, детективы, комедии"
2,avatar_comedy,Голый пистолет,1734,комедии
3,avatar_comedy,Убойные каникулы,4979,"ужасы, комедии"
4,avatar_comedy,"Карты, деньги, два ствола",2866,комедии


In [10]:
train = pd.concat([train, avatar_interactions_action, avatar_interactions_comedy], sort=False)

## Обучение модели

In [11]:
lfm_dataset = LFMDataset()
lfm_dataset.fit(
    users=train["user_id"].values,
    items=train["item_id"].values,
)

train_matrix, _ = lfm_dataset.build_interactions(zip(*train[["user_id", "item_id"]].values.T))

In [12]:
lfm_model = LightFM(
    learning_rate=0.01, 
    loss='warp', 
    no_components=64,
    random_state=RANDOM_STATE
)
lfm_model.fit(
    interactions=train_matrix, 
    epochs=15,
    num_threads=20
);

## Рекомендации для аватаров

In [13]:
n_recommendations = 10

In [14]:
id_item_mapping = {v: k for k, v in lfm_dataset._item_id_mapping.items()}

In [15]:
def get_n_recommendations_for_user(
    user_id: str,
    model: LightFM,
    train_matrix: coo_matrix,
    user_to_id: tp.Dict[str, int],
    id_to_item: tp.Dict[int, str],
    n_recommendations: int
) -> pd.DataFrame:
    user_inner_id = user_to_id[user_id]
    scores = model.predict(
        user_ids=user_inner_id,
        item_ids=np.arange(train_matrix.shape[1]),
        num_threads=20
    )
    user_watched_items = train_matrix.col[train_matrix.row == user_inner_id]
    scores[user_watched_items] = -np.inf

    recommended_item_inner_ids = np.argpartition(scores, -np.arange(n_recommendations))[
        -n_recommendations:
    ][::-1]
    recommended_item_ids = [id_to_item[x] for x in recommended_item_inner_ids]
    return recommended_item_ids


In [16]:
user_id = "avatar_action"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_action,9728,Гнев человеческий,"боевики, триллеры"
1,avatar_action,13865,Девятаев,"драмы, военные, приключения"
2,avatar_action,10440,Хрустальный,"триллеры, детективы"
3,avatar_action,12173,Мстители: Финал,"боевики, драмы, фантастика"
4,avatar_action,7626,Мстители: Война бесконечности,"боевики, фантастика, приключения"
5,avatar_action,14317,Веном,"популярное, фантастика, триллеры, боевики, ужасы"
6,avatar_action,3734,Прабабушка легкого поведения,комедии
7,avatar_action,10942,Мстители,"боевики, фантастика, фэнтези, приключения"
8,avatar_action,5693,Алита: Боевой ангел,"боевики, фантастика, триллеры, приключения"
9,avatar_action,1287,Терминатор: Тёмные судьбы,"боевики, фантастика, приключения"


In [17]:
user_id = "avatar_comedy"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_comedy,9728,Гнев человеческий,"боевики, триллеры"
1,avatar_comedy,10440,Хрустальный,"триллеры, детективы"
2,avatar_comedy,13865,Девятаев,"драмы, военные, приключения"
3,avatar_comedy,15297,Клиника счастья,"драмы, мелодрамы"
4,avatar_comedy,3734,Прабабушка легкого поведения,комедии
5,avatar_comedy,2657,Подслушано,"драмы, триллеры"
6,avatar_comedy,7571,100% волк,"мультфильм, приключения, семейное, фэнтези, ко..."
7,avatar_comedy,4151,Секреты семейной жизни,комедии
8,avatar_comedy,16166,Зверополис,"приключения, мультфильм, детективы, комедии"
9,avatar_comedy,4740,Сахаров. Две жизни,документальное


In [18]:
# самые просматриваемые в обучающей выборке
train["title"].value_counts().head(10)

Хрустальный                     189530
Клиника счастья                 189262
Гнев человеческий               130520
Девятаев                        125905
Прабабушка легкого поведения     74037
Подслушано                       68194
Секреты семейной жизни           66084
Фемида видит                     37020
Афера                            36193
Сахаров. Две жизни               33980
Name: title, dtype: int64

Мы получили, что рекомендации для аватаров имеют сильное пересечение, обусловленное перекосом к рекомендациям популярного контента.

##  Попробуем побороться с перекосом к популярным

Алгоритм обучает для каждого пользователя $u$ и товара $i$ соответственно смещения $b_u$, $b_i$ и эмбеддинги $p_u$, $q_i$. Для формирования рекомендаций для пользователя выбираются товары, имеющие наибольшие значения скоров, определяющихся по формуле:

$$score_{ui} = b_u + b_i + p_u \cdot q_i = b_u + b_i + \cos ( p_u, q_i ) \cdot || p_u || \cdot || q_i || .$$

Часто перекос к популярным выражается в больших значениях смещений или норм эмбеддингов у популярных товаров. В таких случаях может помочь переход от ранжирования по значениям скалярных произведений к ранжированию по косинусам угла между эмбеддингами пользователей и товаров.

Для перехода к косинусам согласно формуле выше достаточно заменить $b_u$ и $b_i$ нулями и привести нормы $p_u$ и $q_i$ к единицам.

In [19]:
lfm_model_cos = deepcopy(lfm_model)

lfm_model_cos.item_biases = np.zeros_like(lfm_model_cos.item_biases)
lfm_model_cos.user_biases = np.zeros_like(lfm_model_cos.user_biases)

lfm_model_cos.item_embeddings = normalize(lfm_model_cos.item_embeddings)
lfm_model_cos.user_embeddings = normalize(lfm_model_cos.user_embeddings)

In [20]:
user_id = "avatar_action"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_action,8579,Библиотекарь 3: Проклятие иудовой чаши,"приключения, триллеры, мелодрамы, боевики, фэн..."
1,avatar_action,1857,Сокровище нации: Книга тайн,"боевики, триллеры, приключения"
2,avatar_action,3404,РЭД,"боевики, триллеры, криминал, комедии"
3,avatar_action,12365,Библиотекарь 2: Возвращение в Копи Царя Соломона,"приключения, драмы, триллеры, мелодрамы, боеви..."
4,avatar_action,4754,Геракл: Начало легенды,"боевики, фэнтези"
5,avatar_action,14775,Район №9,"боевики, фантастика, триллеры"
6,avatar_action,8430,Армагеддон,"боевики, триллеры"
7,avatar_action,6798,Братья из Гримсби,"криминал, приключения, триллеры, боевики, комедии"
8,avatar_action,10058,Элизиум: Рай не на Земле,"боевики, драмы, фантастика"
9,avatar_action,3258,Смертельная гонка 4: Вне анархии,боевики


In [21]:
user_id = "avatar_comedy"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_comedy,7660,Ужастики,"ужасы, фэнтези, приключения, комедии"
1,avatar_comedy,8923,Сердце дракона,"боевики, фэнтези, приключения"
2,avatar_comedy,2543,К-9: Собачья работа,"боевики, криминал, комедии"
3,avatar_comedy,834,Хоттабыч,"фантастика, приключения, комедии"
4,avatar_comedy,11239,Империя мертвых,"фантастика, аниме"
5,avatar_comedy,11761,Ужастики 2: Беспокойный Хэллоуин,"ужасы, фэнтези, приключения, комедии"
6,avatar_comedy,3012,Назад в будущее 2,"для детей, приключения, фантастика, боевики, с..."
7,avatar_comedy,11903,Назад в будущее 3,"фантастика, приключения, комедии"
8,avatar_comedy,15425,Пиксели,"боевики, фантастика, комедии"
9,avatar_comedy,2817,Охотники за привидениями,"боевики, фантастика, фэнтези, комедии"


Видно, что рекомендации стали более персонализированными для аватаров и исчезло преобладание популярного контента. Однако говорить о том, что новая версия модели лучше рано. Выводы стоит делать после того, как будут в том числе проведены количественные оценки качества (оффлайн и онлайн метрики).

## Расчет рекомендаций

Сформируем таблицы рекомендаций для пользователей из тестовой выборки.

Далее с помощью этих таблиц будут оценены метрики beyond accuracy. Методологически вернее оценивать данные метрики на всей пользовательской базе, а не только для пользователей, имеющих действия в тестовой выборке. Здесь мы этим пренебрежем для экономии вычислительных ресурсов.

In [22]:
models_dict = {"lfm": lfm_model, "lfm_cos": lfm_model_cos}

In [23]:
recommendations_dict = {}
for model_name, model in models_dict.items():
    recommendations = pd.DataFrame({"user_id": test["user_id"].unique()})
    recommendations["item_id"] = recommendations["user_id"].apply(
        get_n_recommendations_for_user,
        args=(
            model,
            train_matrix,
            lfm_dataset._user_id_mapping,
            id_item_mapping,
            n_recommendations
        ),
    )
    recommendations = recommendations.explode("item_id")
    recommendations["rank"] = recommendations.groupby(["user_id"]).cumcount() + 1
    recommendations_dict[model_name] = recommendations
    

# Оценка beyond accuracy метрик
## Intra-List Diversity

Оценим для обеих моделей среднее разнообразие контента в полках с помощью метрики $ILD$:

$$ ILD = \frac{1}{|R| ( |R| - 1 )} \sum_{i \in R} \sum_{j \in R} d(i, j) . $$

В качестве расстояния $d(i, j)$ используем [расстояние Хэмминга](https://neerc.ifmo.ru/wiki/index.php?title=Расстояние_Хэмминга) между one-hot векторами жанров. Пример расчета:

##### <center>d(10<font color='blue'>1</font>1<font color='blue'>1</font>01, 10<font color='red'>0</font>1<font color='red'>0</font>01) = 2.</center>



Для оценки расстояния Хэмминга между фильмами вытянем список жанров в one-hot векторы.

In [24]:
items[["item_id", "genres"]].head()

,item_id,genres
0,11838,документальное
1,5990,"зарубежные, триллеры"
2,12119,"зарубежные, триллеры, детективы, ужасы"
3,1504,"драмы, зарубежные"
4,6515,"криминал, драмы, зарубежные, триллеры, боевики"


In [25]:
item_genres_one_hot = items[["item_id", "genres"]].copy()
item_genres_one_hot["genres"] = item_genres_one_hot["genres"].str.split(", ")
item_genres_one_hot = item_genres_one_hot.explode("genres")
item_genres_one_hot["genres"] = item_genres_one_hot["genres"].str.replace(" ", "_")
item_genres_one_hot["genres"] = item_genres_one_hot["genres"].map(lambda x: translit(x, "ru", reversed=True))
item_genres_one_hot["value"] = 1
item_genres_one_hot = item_genres_one_hot.pivot(
    index="item_id", 
    columns="genres", 
    values="value"
).fillna(0).astype(int)

item_genres_one_hot.head()

genres,18+,animatsija,anime,arthaus,biografija,bloger,boeviki,detektivy,detskie,detskie_pesni,...,trillery,uvlechenija,uzhasy,vestern,voennye,vokrug_sveta,vospitanie_detej,zapadnye_mul'tfil'my,zarubezhnye,zhivaja_priroda
item_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
def get_hamming_distances(pairs: pd.Series, features: pd.DataFrame) -> np.ndarray:
    items_0 = pairs.map(lambda pair: pair[1]).values
    items_1 = pairs.map(lambda pair: pair[0]).values

    features_0 = features.reindex(items_0).values
    features_1 = features.reindex(items_1).values
    return np.sum(features_0 != features_1, axis=1)


def calculate_intra_list_diversity_per_user(recommendations: pd.DataFrame, features: pd.DataFrame) -> pd.Series:
    recommended_item_pairs = recommendations.groupby("user_id")["item_id"].apply(
        lambda x: list(combinations(x, 2))
    ).reset_index().explode("item_id").rename(columns={"item_id": "item_pair"})
    recommended_item_pairs["dist"] = get_hamming_distances(recommended_item_pairs["item_pair"], features)
    return recommended_item_pairs[["user_id", "dist"]].groupby("user_id").agg("mean")


In [27]:
for model_name, recommendations in recommendations_dict.items():
    ild_per_user = calculate_intra_list_diversity_per_user(recommendations, item_genres_one_hot)
    print(f"model: {model_name}, mean ild: {round(float(ild_per_user.mean()), 2)}\n")
    

model: lfm, mean ild: 3.21

model: lfm_cos, mean ild: 3.04



Модель, тяготеющая к рекомендации популярного контента, отдает более разнообразные полки по набору жанров. Это логично, т.к. подборки популярного контента обычно разнообразны по значениям различных наборов признаков.

## Mean Inverse User Frequency

Оценим новизну рекомендаций. Новизна товара обратно пропорциональна количеству пользователей, которые с ним взаимодействовали в обучающей выборке. Значение метрики для полки определяется как средняя "новизна" товаров в полке.

$$ MIUF = -\frac{1}{|R|} \sum_{i \in R} \log_2 \frac{|U_i|}{|U|} $$

In [28]:
def calculate_mean_inv_user_frequency_per_user(recommendations: pd.DataFrame, train: pd.DataFrame) -> pd.Series:
    n_users = train["user_id"].nunique()
    n_users_per_item = train.groupby("item_id")["user_id"].nunique()
    
    recommendations_ = recommendations[["user_id", "item_id"]].copy()
    recommendations_["n_users_per_item"] = recommendations_["item_id"].map(n_users_per_item)
    recommendations_["inv_user_freq"] = -np.log2(recommendations_["n_users_per_item"] / n_users)
    return recommendations_[["user_id", "inv_user_freq"]].groupby("user_id").agg("mean")

In [29]:
for model_name, recommendations in recommendations_dict.items():
    miuf_per_user = calculate_mean_inv_user_frequency_per_user(recommendations, train)
    print(f"model: {model_name}, mean miuf: {round(float(miuf_per_user.mean()), 2)}\n")

model: lfm, mean miuf: 4.33

model: lfm_cos, mean miuf: 10.12



Естественно, модель, отдающая полки с популярным контентом, имеет значительно меньшее значение метрики, т.к. метрика принимает высокие значения в тех случаях, когда подборка состоит из контента в "длинном" хвосте.

## Serendipity

Оценим "сюрпризность" рекомендаций, т.е. комбинацию оценки новизны и релевантности контента для пользователя.

$$ Serendipity = \frac{1}{|R|} \sum_{i \in R} \max \left( P_i - P^{U}_i , 0 \right) \cdot rel_i $$

$$ P_i = \frac{|I| + 1 - rank_i}{|I|}; \, P^{U}_i = \frac{|I| + 1 - rank^{U}_i}{|I|} $$

In [30]:
def get_value_popularity_ranks(values: pd.Series) -> pd.Series:
    value_counts = values.value_counts()
    counts_unique = value_counts.unique()
    count_rank_mapping = pd.Series(index=counts_unique, data=np.arange(len(counts_unique)) + 1)
    return value_counts.map(count_rank_mapping)


def calculate_serendipity_per_user(
    recommendations: pd.DataFrame,
    train: pd.DataFrame,
    test: pd.DataFrame,
) -> pd.Series:
    recommendations_ = pd.merge(recommendations, test[["user_id", "item_id"]], how="left", indicator=True)
    recommendations_["is_rel"] = np.where(recommendations_["_merge"] == "both", 1, 0)

    n_items = train["item_id"].nunique()
    item_popularity_ranks = get_value_popularity_ranks(train["item_id"])
    recommendations_["rank_pop"] = recommendations_["item_id"].map(item_popularity_ranks)

    recommendations_["proba_user"] = (n_items + 1 - recommendations_["rank"]) / n_items
    recommendations_["proba_any_user"] = (n_items + 1 - recommendations_["rank_pop"]) / n_items

    recommendations_["proba_diff"] = np.maximum(
        recommendations_["proba_user"] - recommendations_["proba_any_user"],
        0.0
    )
    recommendations_["item_serendipity"] = recommendations_["proba_diff"] * recommendations_["is_rel"]
    return recommendations_[["user_id", "item_serendipity"]].groupby("user_id").agg("mean")


In [31]:
for model_name, recommendations in recommendations_dict.items():
    serendipity_per_user = calculate_serendipity_per_user(recommendations, train, test)
    print(f"model: {model_name}, mean serendipity: {'{0:.06f}'.format(float(serendipity_per_user.mean()))}\n")

model: lfm, mean serendipity: 0.000024

model: lfm_cos, mean serendipity: 0.000137



Значение $Serendipity$ для модели с ранжированием по косинусам оказалось выше, что логично из-за того, что для модели, склонной к рекомендациям популярного контента, множитель c разницами вероятностей часто будет равен или очень близок к нулю.